#지문을 읽고 주제 분류하기

In [ ]:
# 필요한 모듈 임포트
import tensorflow as tf
from keras.layers import Input, Dense, LSTM, Embedding
from keras.models import Model
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

##데이터

In [ ]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
df = pd.DataFrame(paragraph_dict_list)
df = df[['paragraph', 'category']]

In [ ]:
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [ ]:
df.tail()

,paragraph,category
15,the perseyside outfit finished in fourth place...,sports
16,liverpool fc will return to premier league act...,sports
17,alisson signed for liverpool fc from as roma t...,sports
18,but the rankings during that run-in to new yor...,sports
19,then came the oh-so-familiar djokovic-nadal no...,sports


##데이터 전처리

In [ ]:
def get_vocab_size(df):
    """
    데이터에 사용된 중복 없는 전체 단어 갯수를 리턴합니다.
    """
    results = set()
    df['paragraph'].str.lower().str.split().apply(results.update)
    return len(results)

In [ ]:
# 전체 단어 갯수
vocab_size = get_vocab_size(df)
print(vocab_size)

536


In [ ]:
# 단어를 숫자로 인코딩합니다.
paragraphs = df['paragraph'].tolist()
encoded_paragraphs = [one_hot(paragraph, vocab_size) for paragraph in paragraphs]

In [ ]:
def get_max_length(df):
    """
    데이터에서 가장 긴 문장의 단어 갯수를 리턴합니다.
    """
    max_length = 0
    for row in df['paragraph']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length
        
max_length = get_max_length(df)
print(max_length)

91


In [ ]:
# 문장마다 단어갯수가 다르므로, 제로 패딩을 넣어줘서 문장의 길이를 동일하게 만듭니다.
padded_paragraphs_encoding = pad_sequences(encoded_paragraphs, maxlen=max_length, padding='post')

In [ ]:
padded_paragraphs_encoding

array([[450,  75, 496, ...,   0,   0,   0],
       [502, 197, 493, ...,   0,   0,   0],
       [ 90, 144,  38, ...,   0,   0,   0],
       ...,
       [194, 396,  18, ...,   0,   0,   0],
       [302, 377, 375, ...,   0,   0,   0],
       [285,  26, 377, ...,   0,   0,   0]], dtype=int32)

In [ ]:
categories = df['category'].tolist()
def category_encode(category):
    """
    분류 항목(food, sports) 역시 수치로 변경해야합니다. 분류 항목은 one hot encoding으로 변경합니다.
    """
    if category == 'food':
        return [1,0]
    else:
        return [0,1]
encoded_category = [category_encode(category) for category in categories]

In [ ]:
# 원 핫 인코딩 예시
print(encoded_paragraphs[0])
print(encoded_paragraphs[19])

[450, 75, 496, 360, 169, 300, 227, 75, 83, 457, 377, 276, 302, 166, 197, 493, 187, 490, 196, 340, 323, 491, 315, 304, 453, 151, 366, 26, 490, 211, 154, 18, 242, 418, 308, 30, 42, 316, 8, 249, 140, 366, 124, 442, 420, 237, 348, 50, 473, 377, 411, 490, 153]
[285, 26, 377, 442, 140, 220, 200, 245, 199, 519, 395, 56, 18, 461, 360, 377, 483, 294, 227, 316, 301, 499, 206, 259, 473, 490, 377, 497, 259, 310, 354, 242, 393, 230, 409, 245, 437, 349, 236, 261, 201, 308, 224, 227, 151, 322, 393, 378, 490, 482, 377, 301, 389, 302, 166, 516, 200, 426, 41, 275, 249, 91, 382, 146, 75, 210, 10, 245, 18, 308, 415, 327, 302]


##모델 생성

In [ ]:
"""
    Embedding 레이어

    input : 입력될 총 단어 수는 vocab_size(536)
    output : 임베딩 후 출력되는 벡터 크기는 5
    input_length : 단어를 매번 얼마나 입력할지를 추가로 지정

    총 입력되는 단어 수는 vocab_size(536)개이지만 매번 max_length(91)개씩만 넣겠다는 뜻
"""
model = Sequential()
model.add(Embedding(vocab_size, 5, input_length=max_length))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 91, 5)             2680      
                                                                 
 lstm (LSTM)                 (None, 64)                17920     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 22,746
Trainable params: 22,746
Non-trainable params: 0
_________________________________________________________________


##모델 학습

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
train_X = np.array(padded_paragraphs_encoding)
train_Y = np.array(encoded_category)

In [ ]:
print('Train...')
model.fit(train_X, train_Y,batch_size=10,epochs=50,verbose=0)

Train...


##학습정확도

In [ ]:
score, acc = model.evaluate(train_X, train_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

1/1 - 0s - loss: 1.1387e-04 - accuracy: 1.0000 - 26ms/epoch - 26ms/step
Test score: 0.0001138666266342625
Test accuracy: 1.0


##모델 테스트

In [ ]:
print(model.predict(train_X))

[[0.9962782  0.00372175]
 [0.9972836  0.00271638]
 [0.99701536 0.00298465]
 [0.9966317  0.00336831]
 [0.9944794  0.00552052]
 [0.99720484 0.0027951 ]
 [0.9970547  0.00294533]
 [0.9968184  0.00318154]
 [0.9966487  0.00335132]
 [0.99656785 0.00343213]
 [0.00493948 0.9950605 ]
 [0.00403205 0.995968  ]
 [0.00376925 0.9962308 ]
 [0.00487973 0.9951202 ]
 [0.00487762 0.9951224 ]
 [0.00509328 0.9949067 ]
 [0.00514949 0.99485046]
 [0.00538957 0.9946104 ]
 [0.0049569  0.99504304]
 [0.00484696 0.99515307]]
